In [1]:
import numpy as np
import pandas as pd
import matplotlib 
import time
import json
import requests
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import boxscoreadvancedv2, TeamGameLog, TeamGameLogs, leaguegamefinder

The goal of this project is to collect the box scores and advanced stats of past nba games, find the stats with most correlation with number of wins. calculate a rolling average of those stats, and use the resulting data on a model to predict the winner of a game.

## Part A: Data Collection

In [15]:
season = '2022-23'
game_finder = game_finder = leaguegamefinder.LeagueGameFinder(
        player_or_team_abbreviation = 'T',
        season_nullable=season,
        season_type_nullable='Regular Season',
        league_id_nullable='00'  # NBA league ID
)
games_df = game_finder.get_data_frames()[0]
games_df.head(10)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612751,BKN,Brooklyn Nets,0022201217,2023-04-09,BKN vs. PHI,L,240,105,...,0.821,10,32,42,22,6,6,18,22,-29.0
1,22022,1610612759,SAS,San Antonio Spurs,0022201224,2023-04-09,SAS @ DAL,W,240,138,...,0.700,13,56,69,31,2,3,12,13,21.0
2,22022,1610612756,PHX,Phoenix Suns,0022201229,2023-04-09,PHX vs. LAC,L,240,114,...,0.727,12,35,47,29,4,3,7,21,-5.0
3,22022,1610612750,MIN,Minnesota Timberwolves,0022201225,2023-04-09,MIN vs. NOP,W,239,113,...,0.710,8,34,42,25,7,8,17,22,5.0
4,22022,1610612765,DET,Detroit Pistons,0022201223,2023-04-09,DET @ CHI,L,240,81,...,0.769,15,44,59,20,1,3,24,16,-22.0
5,22022,1610612743,DEN,Denver Nuggets,0022201227,2023-04-09,DEN vs. SAC,W,240,109,...,0.720,15,36,51,25,11,2,16,15,14.0
6,22022,1610612752,NYK,New York Knicks,0022201220,2023-04-09,NYK vs. IND,L,241,136,...,0.773,19,34,53,29,8,8,15,24,-5.0
7,22022,1610612748,MIA,Miami Heat,0022201219,2023-04-09,MIA vs. ORL,W,241,123,...,0.750,7,37,44,30,10,3,18,20,13.0
8,22022,1610612757,POR,Portland Trail Blazers,0022201230,2023-04-09,POR vs. GSW,L,241,101,...,0.667,6,24,30,25,5,3,14,9,-56.0
9,22022,1610612741,CHI,Chicago Bulls,0022201223,2023-04-09,CHI vs. DET,W,240,103,...,0.773,11,37,48,21,15,7,3,16,22.0


In [3]:
games_df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [5]:
games_df.isnull().sum()

SEASON_ID            0
TEAM_ID              0
TEAM_ABBREVIATION    0
TEAM_NAME            0
GAME_ID              0
GAME_DATE            0
MATCHUP              0
WL                   0
MIN                  0
PTS                  0
FGM                  0
FGA                  0
FG_PCT               0
FG3M                 0
FG3A                 0
FG3_PCT              0
FTM                  0
FTA                  0
FT_PCT               0
OREB                 0
DREB                 0
REB                  0
AST                  0
STL                  0
BLK                  0
TOV                  0
PF                   0
PLUS_MINUS           0
dtype: int64

In [7]:
games_df.dtypes

SEASON_ID             object
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_NAME             object
GAME_ID               object
GAME_DATE             object
MATCHUP               object
WL                    object
MIN                    int64
PTS                    int64
FGM                    int64
FGA                    int64
FG_PCT               float64
FG3M                   int64
FG3A                   int64
FG3_PCT              float64
FTM                    int64
FTA                    int64
FT_PCT               float64
OREB                   int64
DREB                   int64
REB                    int64
AST                    int64
STL                    int64
BLK                    int64
TOV                    int64
PF                     int64
PLUS_MINUS           float64
dtype: object

All of the important features will be numbers except for WL. Lets take a look at the advanced stats df as well.

In [16]:
advanced_stats = TeamGameLogs(season_nullable = season, season_type_nullable = 'Regular Season', measure_type_player_game_logs_nullable = 'Advanced')
advanced_stats_df = advanced_stats.get_data_frames()[0]
advanced_stats_df.head(10)

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,E_OFF_RATING,...,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK,AVAILABLE_FLAG
0,2022-23,1610612756,PHX,Phoenix Suns,0022201229,2023-04-09T00:00:00,PHX vs. LAC,L,48.0,110.0,...,734,1309,1749,1775,51,1901,2051,627,1410,1
1,2022-23,1610612746,LAC,LA Clippers,0022201229,2023-04-09T00:00:00,LAC @ PHX,W,48.0,111.7,...,2007,706,1150,684,329,1672,1892,627,1051,1
2,2022-23,1610612763,MEM,Memphis Grizzlies,0022201226,2023-04-09T00:00:00,MEM @ OKC,L,48.0,99.1,...,1261,1908,1205,2052,700,2151,2260,803,2002,1
3,2022-23,1610612750,MIN,Minnesota Timberwolves,0022201225,2023-04-09T00:00:00,MIN vs. NOP,W,48.0,108.0,...,1223,1850,1590,1849,1848,1053,1047,459,1092,1
4,2022-23,1610612748,MIA,Miami Heat,0022201219,2023-04-09T00:00:00,MIA vs. ORL,W,48.0,119.6,...,316,1635,1103,748,2070,162,200,1049,583,1
5,2022-23,1610612761,TOR,Toronto Raptors,0022201221,2023-04-09T00:00:00,TOR vs. MIL,W,48.0,116.3,...,498,1747,98,563,201,943,1021,917,580,1
6,2022-23,1610612739,CLE,Cleveland Cavaliers,0022201218,2023-04-09T00:00:00,CLE vs. CHA,L,48.0,94.0,...,882,1654,1286,1935,1698,2304,2295,917,2025,1
7,2022-23,1610612765,DET,Detroit Pistons,0022201223,2023-04-09T00:00:00,DET @ CHI,L,48.0,80.4,...,2178,625,487,254,2453,2418,2442,1335,2408,1
8,2022-23,1610612738,BOS,Boston Celtics,0022201216,2023-04-09T00:00:00,BOS vs. ATL,W,48.0,122.2,...,147,1049,1683,1176,1559,395,544,1537,1149,1
9,2022-23,1610612759,SAS,San Antonio Spurs,0022201224,2023-04-09T00:00:00,SAS @ DAL,W,48.0,118.8,...,773,1183,73,78,478,945,1073,13,204,1


In [17]:
advanced_stats_df.columns

Index(['SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'E_OFF_RATING', 'OFF_RATING',
       'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING', 'NET_RATING', 'AST_PCT',
       'AST_TO', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT',
       'EFG_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE',
       'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'OFF_RATING_RANK', 'DEF_RATING_RANK', 'NET_RATING_RANK', 'AST_PCT_RANK',
       'AST_TO_RANK', 'AST_RATIO_RANK', 'OREB_PCT_RANK', 'DREB_PCT_RANK',
       'REB_PCT_RANK', 'TM_TOV_PCT_RANK', 'EFG_PCT_RANK', 'TS_PCT_RANK',
       'PACE_RANK', 'PIE_RANK', 'AVAILABLE_FLAG'],
      dtype='object')

In [19]:
advanced_stats_df = advanced_stats_df.loc[:, ~advanced_stats_df.columns.str.contains('RANK')]
advanced_stats_df.isnull().sum()

SEASON_YEAR          0
TEAM_ID              0
TEAM_ABBREVIATION    0
TEAM_NAME            0
GAME_ID              0
GAME_DATE            0
MATCHUP              0
WL                   0
MIN                  0
E_OFF_RATING         0
OFF_RATING           0
E_DEF_RATING         0
DEF_RATING           0
E_NET_RATING         0
NET_RATING           0
AST_PCT              0
AST_TO               0
AST_RATIO            0
OREB_PCT             0
DREB_PCT             0
REB_PCT              0
TM_TOV_PCT           0
EFG_PCT              0
TS_PCT               0
E_PACE               0
PACE                 0
PACE_PER40           0
POSS                 0
PIE                  0
AVAILABLE_FLAG       0
dtype: int64

We can see that we are working with very clean datasets for both the box scores and advanced stats datasets. Let's try to merge these 2 datasets together and then do some data exploration.

In [20]:
print(games_df.shape[0])
print(advanced_stats_df.shape[0])

2460
2460


Both datasets have 2460 rows, there are 82x30/2 number of games in a season = 1230 games in a season. Which means for every game played, there are 2 rows in both datasets, one for the stats of the home team, one for the stats of the away team. Each game has a unique game_id, and the 2 rows for each game can be separated using TEAM_ABBREVIATION. Lets try merging the 2 datasets on game_id and TEAM_ABBREVIATION

In [24]:
merged_stats_df = pd.merge(games_df, advanced_stats_df, on=['GAME_ID', 'TEAM_ABBREVIATION'])
print(merged_stats_df.columns)
merged_stats_df.head(10)

Index(['SEASON_ID', 'TEAM_ID_x', 'TEAM_ABBREVIATION', 'TEAM_NAME_x', 'GAME_ID',
       'GAME_DATE_x', 'MATCHUP_x', 'WL_x', 'MIN_x', 'PTS', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS',
       'SEASON_YEAR', 'TEAM_ID_y', 'TEAM_NAME_y', 'GAME_DATE_y', 'MATCHUP_y',
       'WL_y', 'MIN_y', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING',
       'DEF_RATING', 'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TO',
       'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT',
       'TS_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE',
       'AVAILABLE_FLAG'],
      dtype='object')


,SEASON_ID,TEAM_ID_x,TEAM_ABBREVIATION,TEAM_NAME_x,GAME_ID,GAME_DATE_x,MATCHUP_x,WL_x,MIN_x,PTS,...,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,AVAILABLE_FLAG
0,22022,1610612751,BKN,Brooklyn Nets,0022201217,2023-04-09,BKN vs. PHI,L,240,105,...,0.436,0.183,0.494,0.551,106.8,104.5,87.08,104,0.374,1
1,22022,1610612759,SAS,San Antonio Spurs,0022201224,2023-04-09,SAS @ DAL,W,240,138,...,0.590,0.107,0.563,0.589,113.6,112.5,93.75,112,0.603,1
2,22022,1610612756,PHX,Phoenix Suns,0022201229,2023-04-09,PHX vs. LAC,L,240,114,...,0.470,0.068,0.495,0.524,105.1,102.5,85.42,103,0.483,1
3,22022,1610612750,MIN,Minnesota Timberwolves,0022201225,2023-04-09,MIN vs. NOP,W,239,113,...,0.465,0.165,0.555,0.591,105.7,103.5,86.25,103,0.513,1
4,22022,1610612765,DET,Detroit Pistons,0022201223,2023-04-09,DET @ CHI,L,240,81,...,0.563,0.253,0.418,0.446,99.2,99.0,82.50,99,0.352,1
5,22022,1610612743,DEN,Denver Nuggets,0022201227,2023-04-09,DEN vs. SAC,W,240,109,...,0.615,0.206,0.542,0.574,98.3,96.5,80.42,97,0.594,1
6,22022,1610612752,NYK,New York Knicks,0022201220,2023-04-09,NYK vs. IND,L,241,136,...,0.557,0.135,0.551,0.578,113.2,111.0,92.50,111,0.469,1
7,22022,1610612748,MIA,Miami Heat,0022201219,2023-04-09,MIA vs. ORL,W,241,123,...,0.526,0.178,0.651,0.670,102.2,100.5,83.75,101,0.556,1
8,22022,1610612757,POR,Portland Trail Blazers,0022201230,2023-04-09,POR vs. GSW,L,241,101,...,0.404,0.125,0.447,0.478,111.8,111.5,92.92,112,0.264,1
9,22022,1610612741,CHI,Chicago Bulls,0022201223,2023-04-09,CHI vs. DET,W,240,103,...,0.438,0.040,0.453,0.492,99.2,99.0,82.50,99,0.648,1


Now that the datasets are merged, there are some duplicate columns that needs to be dropped.

In [27]:
merged_stats_df = merged_stats_df.drop(columns=[col for col in merged_stats_df.columns if col.endswith('_y')])
merged_stats_df = merged_stats_df.rename(columns={
    'TEAM_ID_x': 'TEAM_ID',
    'TEAM_NAME_x': 'TEAM_NAME',
    'GAME_DATE_x': 'GAME_DATE',
    'MATCHUP_x': 'MATCHUP',
    'WL_X': 'WL',
    'MIN_x': 'MIN'
})
print(merged_stats_df.columns)

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL_x', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS', 'SEASON_YEAR',
       'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TO', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE', 'AVAILABLE_FLAG'],
      dtype='object')


Now that the data is merged, let's put it in a function so that we can run a for loop to gather multiple seasons worth of data instead of just one season.

In [ ]:
def get_season_games(season, season_type="Regular Season"):

    # Create the game finder object
    game_finder = leaguegamefinder.LeagueGameFinder(
        player_or_team_abbreviation = 'T',
        season_nullable=season,
        season_type_nullable=season_type,
        league_id_nullable='00'  # NBA league ID
    )
    
    # Get the games dataframe
    games_df = game_finder.get_data_frames()[0]
    
    return games_df

#using teamGameLogs
def get_advanced_stats(season):
    advanced_stats = TeamGameLogs(season_nullable = season, season_type_nullable = 'Regular Season', measure_type_player_game_logs_nullable = 'Advanced')
    df = advanced_stats.get_data_frames()[0]
    df = df.loc[:, ~df.columns.str.contains('RANK')]
    return df

def get_multiple_seasons_stats(start_year, end_year, season_type="Regular Season"):
    all_games_df = pd.DataFrame()
    
    for i in range(start_year, end_year):
        time.sleep(1)
        current_season = str(i) + "-" + str(i+1)[2:]
        games_season = get_season_games(current_season, season_type=season_type)
        advanced_season = get_advanced_stats(current_season)

        games_season = games_season.sort_values(by=['MATCHUP', 'GAME_DATE'])
        advanced_season = advanced_season.sort_values(by=['MATCHUP', 'GAME_DATE'])
        merged_stats_df = pd.merge(games_season, advanced_season, on=['GAME_ID', 'TEAM_ABBREVIATION'])
        merged_stats_df = merged_stats_df.drop(columns=[col for col in merged_stats_df.columns if col.endswith('_y')])
        merged_stats_df = merged_stats_df.rename(columns={
            'TEAM_ID_x': 'TEAM_ID',
            'TEAM_NAME_x': 'TEAM_NAME',
            'GAME_DATE_x': 'GAME_DATE',
            'MATCHUP_x': 'MATCHUP',
            'WL_x': 'WL',
            'MIN_x': 'MIN'
        })

        all_games_df = pd.concat([all_games_df, merged_stats_df])
    all_games_df = all_games_df.sort_values(by='GAME_DATE')
    return all_games_df
start_year = 2018
end_year = 2024
all_games_df = get_multiple_seasons_stats(start_year = start_year, end_year = end_year)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,AVAILABLE_FLAG
806,22018,1610612744,GSW,Golden State Warriors,0021800002,2018-10-16,GSW vs. OKC,W,241,108,...,0.546,0.204,0.479,0.525,106.6,103.0,85.83,103,0.582,1
1651,22018,1610612760,OKC,Oklahoma City Thunder,0021800002,2018-10-16,OKC @ GSW,L,240,100,...,0.454,0.146,0.418,0.466,106.6,103.0,85.83,103,0.418,1
236,22018,1610612738,BOS,Boston Celtics,0021800001,2018-10-16,BOS vs. PHI,W,240,105,...,0.540,0.143,0.490,0.509,106.6,105.5,87.92,105,0.595,1
1808,22018,1610612755,PHI,Philadelphia 76ers,0021800001,2018-10-16,PHI @ BOS,L,239,87,...,0.460,0.151,0.420,0.448,106.6,105.5,87.92,106,0.405,1
1785,22018,1610612753,ORL,Orlando Magic,0021800006,2018-10-17,ORL vs. MIA,W,240,104,...,0.488,0.117,0.452,0.483,107.7,103.0,85.83,103,0.560,1
522,22018,1610612742,DAL,Dallas Mavericks,0021800013,2018-10-17,DAL @ PHX,L,240,100,...,0.489,0.095,0.489,0.517,94.9,95.0,79.17,95,0.408,1
1318,22018,1610612749,MIL,Milwaukee Bucks,0021800003,2018-10-17,MIL @ CHA,W,240,113,...,0.583,0.204,0.576,0.602,103.7,103.5,86.25,103,0.527,1
2264,22018,1610612761,TOR,Toronto Raptors,0021800008,2018-10-17,TOR vs. CLE,W,239,116,...,0.436,0.100,0.565,0.575,103.5,100.0,83.33,100,0.585,1
1033,22018,1610612746,LAC,LA Clippers,0021800012,2018-10-17,LAC vs. DEN,L,240,98,...,0.455,0.133,0.443,0.497,103.0,104.5,87.08,105,0.422,1
1599,22018,1610612752,NYK,New York Knicks,0021800007,2018-10-17,NYK vs. ATL,W,240,126,...,0.523,0.143,0.515,0.559,116.8,112.5,93.75,112,0.595,1


In [33]:
print(all_games_df['GAME_ID'].nunique())
all_games_df.head(10)


7059


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,AVAILABLE_FLAG
806,22018,1610612744,GSW,Golden State Warriors,0021800002,2018-10-16,GSW vs. OKC,W,241,108,...,0.546,0.204,0.479,0.525,106.6,103.0,85.83,103,0.582,1
1651,22018,1610612760,OKC,Oklahoma City Thunder,0021800002,2018-10-16,OKC @ GSW,L,240,100,...,0.454,0.146,0.418,0.466,106.6,103.0,85.83,103,0.418,1
236,22018,1610612738,BOS,Boston Celtics,0021800001,2018-10-16,BOS vs. PHI,W,240,105,...,0.540,0.143,0.490,0.509,106.6,105.5,87.92,105,0.595,1
1808,22018,1610612755,PHI,Philadelphia 76ers,0021800001,2018-10-16,PHI @ BOS,L,239,87,...,0.460,0.151,0.420,0.448,106.6,105.5,87.92,106,0.405,1
1785,22018,1610612753,ORL,Orlando Magic,0021800006,2018-10-17,ORL vs. MIA,W,240,104,...,0.488,0.117,0.452,0.483,107.7,103.0,85.83,103,0.560,1
522,22018,1610612742,DAL,Dallas Mavericks,0021800013,2018-10-17,DAL @ PHX,L,240,100,...,0.489,0.095,0.489,0.517,94.9,95.0,79.17,95,0.408,1
1318,22018,1610612749,MIL,Milwaukee Bucks,0021800003,2018-10-17,MIL @ CHA,W,240,113,...,0.583,0.204,0.576,0.602,103.7,103.5,86.25,103,0.527,1
2264,22018,1610612761,TOR,Toronto Raptors,0021800008,2018-10-17,TOR vs. CLE,W,239,116,...,0.436,0.100,0.565,0.575,103.5,100.0,83.33,100,0.585,1
1033,22018,1610612746,LAC,LA Clippers,0021800012,2018-10-17,LAC vs. DEN,L,240,98,...,0.455,0.133,0.443,0.497,103.0,104.5,87.08,105,0.422,1
1599,22018,1610612752,NYK,New York Knicks,0021800007,2018-10-17,NYK vs. ATL,W,240,126,...,0.523,0.143,0.515,0.559,116.8,112.5,93.75,112,0.595,1


Now we have collected 6 seasons worth of games, which gives us 7059 unique games. Let's do some EDA

## Part B: Data Exploration

In [34]:
all_games_df.describe()

,TEAM_ID,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,AVAILABLE_FLAG
count,1.411800e+04,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,...,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000,14118.000000
mean,1.610613e+09,241.626718,112.457855,41.334537,88.624451,0.467380,12.309109,34.202507,0.359168,17.479671,...,0.500003,0.138958,0.537096,0.571861,101.578892,99.803423,83.169511,100.446026,0.500003,1.002550
std,8.646615e+00,6.933706,12.600808,5.228212,7.063444,0.054901,3.878339,7.001189,0.084795,5.927496,...,0.049580,0.037633,0.066019,0.061651,4.746175,4.681078,3.900848,5.391221,0.083490,0.050434
min,1.610613e+09,237.000000,68.000000,21.000000,63.000000,0.269000,2.000000,10.000000,0.069000,0.000000,...,0.310000,0.011000,0.301000,0.356000,85.200000,83.500000,69.580000,83.000000,0.134000,1.000000
25%,1.610613e+09,240.000000,104.000000,38.000000,84.000000,0.430000,10.000000,29.000000,0.302000,13.000000,...,0.467000,0.112000,0.490000,0.530000,98.400000,96.500000,80.420000,97.000000,0.442000,1.000000
50%,1.610613e+09,240.000000,112.000000,41.000000,88.000000,0.467000,12.000000,34.000000,0.357000,17.000000,...,0.500000,0.137000,0.535000,0.570000,101.500000,99.500000,82.920000,100.000000,0.500000,1.000000
75%,1.610613e+09,241.000000,121.000000,45.000000,93.000000,0.505000,15.000000,39.000000,0.415000,21.000000,...,0.533000,0.163000,0.580000,0.614000,104.700000,103.000000,85.830000,104.000000,0.558000,1.000000
max,1.610613e+09,341.000000,176.000000,65.000000,123.000000,0.687000,29.000000,70.000000,0.842000,44.000000,...,0.690000,0.309000,0.795000,0.808000,121.300000,119.500000,99.580000,145.000000,0.866000,2.000000


In [4]:
# get averages of the following stats of  a team prior to a game:
# - OFF_RATING, DEF_RATING, NET_RATING, EFG_PCT, TS_PCT, PIE
# - num_games: number of games to consider before the game
# - game_id: the game to consider
# - team_abbreviation: the team to consider
def get_rolling_averages(stats_pd, column_names, num_games=10):
    rolling_average_pd = stats_pd.loc[:, ['GAME_DATE','GAME_ID', 'TEAM_ABBREVIATION']].copy()
    for team_id, stats in stats_pd.groupby(['TEAM_ID']):
        stats = stats.sort_values(by='GAME_DATE')
        for column in column_names:
            rolling_average_pd.loc[stats.index, column] = stats[column].shift(1).rolling(window=num_games, min_periods=10).mean().round(5)
    rolling_average_pd = rolling_average_pd.drop(columns='GAME_DATE')
    return rolling_average_pd

In [5]:
feature_names = ['NET_RATING', 'TS_PCT', 'PIE', '3PT_PCT']
start_year = 2014
end_year = 2024

In [6]:
def get_all_stats(start_year, end_year, features, season_type="Regular Season"):
    all_games_df = pd.DataFrame()
    advanced_df = pd.DataFrame()
    advanced_rolling_df = pd.DataFrame()
    
    for i in range(start_year, end_year):
        time.sleep(1)
        season = str(i) + "-" + str(i+1)[2:]
        print(season)
        games_season = get_season_games(season=season, season_type=season_type)
        advanced_season = get_advanced_stats(season)

        games_season = games_season.sort_values(by=['MATCHUP', 'GAME_DATE'])
        advanced_season = advanced_season.sort_values(by=['MATCHUP', 'GAME_DATE'])
        advanced_season['3PT_PCT'] = games_season['FG3_PCT']

        rolling_season = get_rolling_averages(advanced_season, feature_names)
        all_games_df = pd.concat([all_games_df, games_season])
        advanced_df = pd.concat([advanced_df, advanced_season])
        advanced_rolling_df = pd.concat([advanced_rolling_df, rolling_season])
    all_games_df = all_games_df.sort_values(by='GAME_DATE')
    advanced_df = advanced_df.sort_values(by=['TEAM_ABBREVIATION', 'GAME_DATE'])

    home_games = all_games_df[~all_games_df['MATCHUP'].str.contains('@')]
    away_games = all_games_df[all_games_df['MATCHUP'].str.contains('@')]
    unique_games = combine_home_away_games(home_games, away_games)
    return unique_games, advanced_df, advanced_rolling_df

unique_games, advanced_stats_df, rolling_df = get_all_stats(start_year, end_year, feature_names)


2014-15


/var/folders/7j/qd067fns3xz9wjgwbvkfz5940000gn/T/ipykernel_14357/2410233092.py:8: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for team_id, stats in stats_pd.groupby(['TEAM_ID']):


2015-16


/var/folders/7j/qd067fns3xz9wjgwbvkfz5940000gn/T/ipykernel_14357/2410233092.py:8: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for team_id, stats in stats_pd.groupby(['TEAM_ID']):


2016-17


...

/var/folders/7j/qd067fns3xz9wjgwbvkfz5940000gn/T/ipykernel_14357/2410233092.py:8: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for team_id, stats in stats_pd.groupby(['TEAM_ID']):


2022-23


/var/folders/7j/qd067fns3xz9wjgwbvkfz5940000gn/T/ipykernel_14357/2410233092.py:8: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for team_id, stats in stats_pd.groupby(['TEAM_ID']):


2023-24


/var/folders/7j/qd067fns3xz9wjgwbvkfz5940000gn/T/ipykernel_14357/2410233092.py:8: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for team_id, stats in stats_pd.groupby(['TEAM_ID']):


In [8]:
unique_games.describe()

,TEAM_ID,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,AWAY_FT_PCT,AWAY_OREB,AWAY_DREB,AWAY_REB,AWAY_AST,AWAY_STL,AWAY_BLK,AWAY_TOV,AWAY_PF,AWAY_PLUS_MINUS
count,1.197900e+04,11979.000000,11979.000000,11979.000000,11979.000000,11979.000000,11979.000000,11979.000000,11979.000000,11979.000000,...,11978.000000,11979.000000,11979.000000,11979.000000,11979.000000,11979.000000,11979.000000,11979.000000,11979.000000,11979.000000
mean,1.610613e+09,241.678688,109.975541,40.570415,87.053510,0.466962,11.138075,30.743134,0.361171,17.696636,...,0.769266,10.191335,33.255781,43.447116,23.565907,7.640287,4.714333,13.503715,20.173804,-2.265598
std,8.651633e+00,7.151068,13.064024,5.325264,7.354049,0.055312,4.099159,8.261164,0.090771,6.019321,...,0.104221,3.769506,5.377596,6.491445,5.186944,2.905149,2.439833,3.878762,4.327452,14.258937
min,1.610613e+09,237.000000,64.000000,19.000000,60.000000,0.269000,0.000000,4.000000,0.000000,1.000000,...,0.176000,0.000000,16.000000,20.000000,6.000000,0.000000,0.000000,2.000000,7.000000,-73.000000
25%,1.610613e+09,240.000000,101.000000,37.000000,82.000000,0.429000,8.000000,25.000000,0.300000,13.000000,...,0.704000,7.000000,30.000000,39.000000,20.000000,6.000000,3.000000,11.000000,17.000000,-11.000000
50%,1.610613e+09,240.000000,110.000000,40.000000,87.000000,0.466000,11.000000,31.000000,0.360000,17.000000,...,0.778000,10.000000,33.000000,43.000000,23.000000,7.000000,4.000000,13.000000,20.000000,-3.000000
75%,1.610613e+09,241.000000,119.000000,44.000000,92.000000,0.505000,14.000000,36.000000,0.419000,22.000000,...,0.842000,13.000000,37.000000,48.000000,27.000000,9.000000,6.000000,16.000000,23.000000,8.000000
max,1.610613e+09,341.000000,175.000000,65.000000,125.000000,0.684000,28.000000,70.000000,0.842000,44.000000,...,1.000000,38.000000,60.000000,81.000000,47.000000,20.000000,19.000000,30.000000,42.000000,57.000000


In [9]:
rolling_df.describe()

,NET_RATING,TS_PCT,PIE,3PT_PCT
count,20958.000000,20958.000000,20958.000000,20958.000000
mean,0.011717,0.562100,0.500086,0.357578
std,6.423475,0.029484,0.038784,0.029194
min,-22.410000,0.465800,0.344100,0.237300
25%,-4.260000,0.541600,0.474100,0.337800
50%,0.110000,0.561600,0.501200,0.357200
75%,4.250000,0.582600,0.525900,0.377000
max,22.350000,0.664400,0.637600,0.483700


In [8]:
features_df = unique_games[['GAME_DATE', 'GAME_ID', 'TEAM_ABBREVIATION', 'WL', 'AWAY_TEAM_ABBREVIATION']].copy()
features_df = features_df.merge(rolling_df, left_on=['GAME_ID', 'TEAM_ABBREVIATION'], right_on=['GAME_ID', 'TEAM_ABBREVIATION'])
away_stats_df = rolling_df.rename(columns={col: f'AWAY_{col}' for col in rolling_df.columns if col != 'GAME_ID'})

features_df = features_df.merge(away_stats_df, left_on=['GAME_ID', 'AWAY_TEAM_ABBREVIATION'], right_on=['GAME_ID', 'AWAY_TEAM_ABBREVIATION'])
print(features_df.columns)
features_df = features_df.loc[:, ~features_df.columns.duplicated()]
features_df = features_df.dropna()
display(features_df)
    

Index(['GAME_DATE', 'GAME_ID', 'TEAM_ABBREVIATION', 'WL',
       'AWAY_TEAM_ABBREVIATION', 'NET_RATING', 'TS_PCT', 'PIE', '3PT_PCT',
       'AWAY_NET_RATING', 'AWAY_TS_PCT', 'AWAY_PIE', 'AWAY_3PT_PCT'],
      dtype='object')


,GAME_DATE,GAME_ID,TEAM_ABBREVIATION,WL,AWAY_TEAM_ABBREVIATION,NET_RATING,TS_PCT,PIE,3PT_PCT,AWAY_NET_RATING,AWAY_TS_PCT,AWAY_PIE,AWAY_3PT_PCT
145,2014-11-17,0021400150,DET,L,ORL,-4.02,0.4970,0.4511,0.3246,-1.62,0.5464,0.4766,0.3650
149,2014-11-17,0021400146,CHA,L,DAL,-3.86,0.5119,0.4889,0.3513,9.73,0.5795,0.5752,0.3485
151,2014-11-17,0021400151,MEM,W,HOU,4.87,0.5276,0.5434,0.3454,9.93,0.5455,0.5433,0.3487
154,2014-11-18,0021400157,UTA,W,OKC,-5.55,0.5598,0.4942,0.3480,-3.97,0.5105,0.4835,0.3467
155,2014-11-18,0021400156,MIL,W,NYK,-0.38,0.5080,0.5071,0.4517,-2.84,0.5353,0.4678,0.3034
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,2024-04-14,0022301186,BOS,W,WAS,9.26,0.5907,0.5466,0.3678,-4.51,0.5712,0.4681,0.3669
11975,2024-04-14,0022301190,NYK,W,CHI,7.82,0.6008,0.5166,0.3546,0.14,0.5786,0.5143,0.3941
11976,2024-04-14,0022301199,LAC,L,HOU,3.51,0.5789,0.5106,0.3043,-5.47,0.5617,0.4631,0.3118
11977,2024-04-14,0022301200,SAC,W,POR,-0.52,0.5527,0.4915,0.3911,-12.56,0.5030,0.4177,0.3620


Classical ML

Predict Wins and Losses of NBA games using classical ML techniques like logistic regression and random forest regression by analyzing advanced NBA stats

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [10]:
features_df['HOME_WIN'] = features_df['WL'].apply(lambda x: 1 if x == 'W' else 0) 
all_features = feature_names + ['AWAY_' + feature_name for feature_name in feature_names]
scaler = StandardScaler()
features_df_scaled = features_df.copy()
features_df_scaled[all_features] = scaler.fit_transform(features_df[all_features])
display(features_df_scaled)


,GAME_DATE,GAME_ID,TEAM_ABBREVIATION,WL,AWAY_TEAM_ABBREVIATION,NET_RATING,TS_PCT,PIE,3PT_PCT,AWAY_NET_RATING,AWAY_TS_PCT,AWAY_PIE,AWAY_3PT_PCT,HOME_WIN
145,2014-11-17,0021400150,DET,L,ORL,-0.613737,-2.203425,-1.247204,-1.129231,-0.268773,-0.543896,-0.621667,0.251540,0
149,2014-11-17,0021400146,CHA,L,DAL,-0.588808,-1.696809,-0.274227,-0.217363,1.497028,0.575746,1.928154,-0.308552,0
151,2014-11-17,0021400151,MEM,W,HOU,0.771366,-1.162992,1.128611,-0.418862,1.528143,-0.574340,1.103212,-0.301763,1
154,2014-11-18,0021400157,UTA,W,OKC,-0.852118,-0.068157,-0.137804,-0.330066,-0.634379,-1.758252,-0.443231,-0.369653,1
155,2014-11-18,0021400156,MIL,W,NYK,-0.046608,-1.829413,0.194244,3.211532,-0.458577,-0.919365,-0.849237,-1.839470,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,2024-04-14,0022301186,BOS,W,WAS,1.455349,0.982477,1.210979,0.346150,-0.718391,0.294990,-0.841479,0.316036,1
11975,2024-04-14,0022301190,NYK,W,CHI,1.230990,1.325888,0.438775,-0.104661,0.005043,0.545303,0.353265,1.239339,1
11976,2024-04-14,0022301199,LAC,L,HOU,0.559472,0.581264,0.284334,-1.822524,-0.867745,-0.026358,-0.970780,-1.554332,0
11977,2024-04-14,0022301200,SAC,W,POR,-0.068421,-0.309564,-0.207302,1.141900,-1.970787,-2.011947,-2.144835,0.149705,1


In [11]:
X = features_df_scaled[all_features]
y = features_df_scaled['HOME_WIN']
# train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, stratify=y)
print(y_train.sum()/y_train.size)

0.5691144708423326


In [12]:
def print_model_results(model, y_pred):
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    print("Cross-validation scores:", cv_scores)
    print("Mean accuracy:", cv_scores.mean())
    display(confusion_matrix(y_true=y_test, y_pred=y_pred))
    print("AUC Score:", roc_auc_score(y_test, y_pred))

In [13]:
logistic_regression_model = LogisticRegressionCV(random_state=42, max_iter = 1000, solver='newton-cg')
logistic_regression_model.fit(X_train, y_train)

y_pred = logistic_regression_model.predict(X_test)
print_model_results(logistic_regression_model, y_pred)


Cross-validation scores: [0.63827235 0.63167367 0.64187163 0.64547091 0.63205282]
Mean accuracy: 0.6378682726839986


array([[394, 504],
       [262, 924]])

AUC Score: 0.6089210800091641


In [14]:
decision_tree_model = DecisionTreeClassifier(random_state=42, max_depth=5, class_weight='balanced')
decision_tree_model.fit(X_train, y_train)
y_pred_1 = decision_tree_model.predict(X_test)
print_model_results(decision_tree_model, y_pred_1)

Cross-validation scores: [0.61787642 0.61907618 0.61427714 0.6124775  0.58463385]
Mean accuracy: 0.6096682224179414


array([[559, 339],
       [511, 675]])

AUC Score: 0.5958171991722283


In [20]:
xgb_model = xgb.XGBClassifier(n_estimators=200, colsample_bytree=0.8, max_depth=5, learning_rate=0.1, objective="binary:logistic")
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
print_model_results(xgb_model, y_pred_xgb)


Cross-validation scores: [0.62987403 0.63047391 0.60827834 0.61907618 0.62484994]
Mean accuracy: 0.6225104798968177


array([[421, 477],
       [307, 879]])

AUC Score: 0.604983155372441


In [18]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

grid_search = GridSearchCV(
    estimator=xgb.XGBClassifier(objective="binary:logistic", eval_metric="logloss"),
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)
y_pred_xgb = grid_search.predict(X_test)
print_model_results(grid_search, y_pred_xgb)

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.7}


Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x136cc3e10>>
Traceback (most recent call last):
  File "/Users/j52zhao/anaconda3/lib/python3.11/site-packages/xgboost/core.py", line 582, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument

KeyboardInterrupt: 


KeyboardInterrupt: 

In [21]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

rf_pred= rf_model.predict(X_test)
print_model_results(rf_model, rf_pred)

Cross-validation scores: [0.63467307 0.64787043 0.63947211 0.63167367 0.62244898]
Mean accuracy: 0.6352276483478815


array([[418, 480],
       [290, 896]])

AUC Score: 0.6104797244767274
